# Dummy Tweet Scraping

### Import Libraries

In [101]:
from tweepy import Stream
from tweepy import OAuthHandler
#from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import string
import time
import datetime
import numpy as np
from cmath import nan

### Account Authentification

In [102]:
# Authenticating Twitter API
# Obtain your Twitter credentials from your twitter developer account

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAEGchQEAAAAAZSYFv1nyLDV81YAKEfDr1fVrlho%3DWKBvyLhQ4CeHrlBRtecAetYkB1ZnAjI3Zydb1516fkIzKhS4vh'

client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

### Search Criteria
This is just an arbitary search criteria for Dummy results with the word protest in them

In [103]:
search_words = '(protest OR strike OR march OR rally OR #feesmustfall OR #zumamustfall) -is:retweet lang:en place_country:ZA'
#search_words = 'protest -is:retweet lang:en place_country:ZA'
date_since = datetime.datetime(2021,1,1)
date_before = datetime.datetime(2022,9,30)
numTweets = 500

Scrape Tweets

In [104]:
# Scrape Tweets
# Note for the final versions we should use -> until_id -> allos to resume scraping from a particualar tweet ID
protest_tweets = []
count = 0 
for response in tweepy.Paginator(client.search_all_tweets, 
                                 query = search_words,
                                 user_fields = ['public_metrics'],
                                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                 place_fields = ['id', 'geo', 'name', 'country_code', 'place_type', 'full_name', 'country'],
                                 expansions = ['author_id', 'geo.place_id'],
                                 start_time = date_since,
                                 end_time = date_before,
                                 max_results=numTweets
                              ):
   time.sleep(1)
   protest_tweets.append(response)

Populate Dataframe with tweet data

In [ ]:
error_count = 0
result = []
user_dict = {}
place_dict = {}
# Loop through each response object
for response in protest_tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes['users']:
        user_dict[user.id] = {'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                             }
    
    place_dict = {p['id']: p for p in response.includes['places']}

    for tweet in response.data:
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet

        try:
            if place_dict[tweet.geo['place_id']]:
                place = place_dict[tweet.geo['place_id']]   
            else:
                place = nan

            result.append({'tweet_id': tweet.id,
                       'author_id': tweet.author_id, 
                       'author_followers': author_info['followers'],
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       'retweets': tweet.public_metrics['retweet_count'],
                       'replies': tweet.public_metrics['reply_count'],
                       'likes': tweet.public_metrics['like_count'],
                       'quote_count': tweet.public_metrics['quote_count'],
                       'place_id': tweet.geo,
                       'place_name': place.full_name,
                       'bbox': place.geo['bbox']

                      })

        except:
            error_count = error_count + 1


print(error_count, " result(s) were excluded due to invalid data points.")        

# Change this list of dictionaries into a dataframe             
df = pd.DataFrame(result)                

5  result(s) were excluded due to invalid data points.


In [ ]:
# (number of tweets, number of attributes)
df.shape

(21686, 12)

Save data to csv

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21686 entries, 0 to 21685
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   tweet_id          21686 non-null  int64              
 1   author_id         21686 non-null  int64              
 2   author_followers  21686 non-null  int64              
 3   text              21686 non-null  object             
 4   created_at        21686 non-null  datetime64[ns, UTC]
 5   retweets          21686 non-null  int64              
 6   replies           21686 non-null  int64              
 7   likes             21686 non-null  int64              
 8   quote_count       21686 non-null  int64              
 9   place_id          21686 non-null  object             
 10  place_name        21686 non-null  object             
 11  bbox              21686 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(7), object(4)
memory usage:

In [ ]:
df.head(2)

,tweet_id,author_id,author_followers,text,created_at,retweets,replies,likes,quote_count,place_id,place_name,bbox
0,1575571139358691329,1353804645580353537,1436,Go shota R1000 ya registration \n\nBlesser: +R...,2022-09-29 19:40:06+00:00,0,0,1,0,{'place_id': 'dd9c0d7d7e07eb49'},South Africa,"[16.4475932, -34.8342468, 32.8922934, -22.1247..."
1,1575489974316584961,1258457287120818182,1326,Spx yesterday's rally\nPoof gone https://t.co/...,2022-09-29 14:17:35+00:00,0,0,1,0,{'place_id': 'a02e6c261fa62b42'},"Benoni, South Africa","[28.2722463, -26.2315204, 28.4449594, -26.0681..."


In [ ]:
df.to_csv('CSV/Scrape_Tweets.csv')